In [1]:
import cv2
import numpy as np
from datetime import datetime

# Placeholder classification function (replace with your actual PyTorch-based function)
def get_classification(frame):
    # Dummy implementation; replace with your model inference
    import random
    return random.choice(["NoFall", "Fall"])

# Constants for window and element positions
CAMERA = 1
WINDOW_WIDTH = 1080
WINDOW_HEIGHT = 1920
VIDEO_WIDTH = 640
VIDEO_HEIGHT = 480
VIDEO_X = (WINDOW_WIDTH - VIDEO_WIDTH) // 2  # Center horizontally
VIDEO_Y = 50  # Top margin
LABEL_Y = VIDEO_Y + VIDEO_HEIGHT + 50  # Below video
BUTTON_WIDTH = 300  # Large button
BUTTON_HEIGHT = 150
BUTTON_X = (WINDOW_WIDTH - BUTTON_WIDTH) // 2  # Center horizontally
BUTTON_Y = WINDOW_HEIGHT - BUTTON_HEIGHT - 50  # Near bottom

# Global variables
paused = False
prev_classification = "NoFall"
fall_detected_time = None
last_frame = None

def on_mouse(event, x, y, flags, param):
    """Handle mouse clicks to toggle pause/resume."""
    global paused
    if event == cv2.EVENT_LBUTTONDOWN:
        if BUTTON_X <= x <= BUTTON_X + BUTTON_WIDTH and BUTTON_Y <= y <= BUTTON_Y + BUTTON_HEIGHT:
            paused = not paused

def main():
    global last_frame, prev_classification, fall_detected_time
    
    # Initialize video capture
    cap = cv2.VideoCapture(CAMERA)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return
    
    # Set up window and mouse callback
    cv2.namedWindow('Video')
    cv2.setMouseCallback('Video', on_mouse)
    
    while True:
        if not paused:
            # Capture and resize frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break
            frame = cv2.resize(frame, (VIDEO_WIDTH, VIDEO_HEIGHT))
            last_frame = frame.copy()
            
            # Classify the frame
            current_classification = get_classification(frame)
            if current_classification == "Fall" and prev_classification == "NoFall":
                fall_detected_time = datetime.now()
            prev_classification = current_classification
        else:
            # Use last frame when paused
            if last_frame is None:
                continue
            frame = last_frame.copy()
            current_classification = prev_classification
        
        # Create blank canvas
        canvas = np.zeros((WINDOW_HEIGHT, WINDOW_WIDTH, 3), dtype=np.uint8)
        
        # Place video frame at the top
        canvas[VIDEO_Y:VIDEO_Y + VIDEO_HEIGHT, VIDEO_X:VIDEO_X + VIDEO_WIDTH] = frame
        
        # Draw classification label
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        thickness = 3
        if current_classification == "Fall" and fall_detected_time is not None:
            text = f"Fall detected at {fall_detected_time.strftime('%H:%M:%S')}"
            color = (0, 0, 255)  # Red
        else:
            text = "NoFall"
            color = (0, 255, 0)  # Green
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = (WINDOW_WIDTH - text_size[0]) // 2  # Center horizontally
        cv2.putText(canvas, text, (text_x, LABEL_Y), font, font_scale, color, thickness)
        
        # Draw large button
        cv2.rectangle(canvas, (BUTTON_X, BUTTON_Y), 
                      (BUTTON_X + BUTTON_WIDTH, BUTTON_Y + BUTTON_HEIGHT), 
                      (255, 0, 0), -1)  # Blue button
        button_text = "Pause" if not paused else "Resume"
        button_font_scale = 1.5
        button_text_size = cv2.getTextSize(button_text, font, button_font_scale, thickness)[0]
        button_text_x = BUTTON_X + (BUTTON_WIDTH - button_text_size[0]) // 2  # Center text
        button_text_y = BUTTON_Y + (BUTTON_HEIGHT + button_text_size[1]) // 2 - 10  # Adjust vertically
        cv2.putText(canvas, button_text, (button_text_x, button_text_y), 
                    font, button_font_scale, (255, 255, 255), thickness)  # White text
        
        # Display the canvas
        cv2.imshow('Video', canvas)
        
        # Exit on 'q' key
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

2025-03-21 12:32:42.789 python[51169:5630832] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
